In [2]:
from asyncore import read
import spacy
from spacy import displacy
from utils.utils import count_occurrences, get_entities, search_with_google, wiki_content
import json

nlp = spacy.load('it_core_news_sm')

with open(f'assets/test_sentences.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

ModuleNotFoundError: No module named 'utils'

In [2]:
sentence_list = list(doc.sents) 

# Opening JSON file
f = open('./assets/italian_cities.json')
italian_cities = json.load(f)
f.close()
# Count occurrences
counter = count_occurrences(doc, italian_cities, "name")  

# Max occurrence
context = max(counter, key=counter.get) 
print(context)

# Get entities without duplicates
searchable_entities = get_entities(doc, counter)

print(searchable_entities)
#print(searchable_entities)

# Search addresses with Google
search_with_google(searchable_entities, context)

Torino
['Novecento', 'Italia', 'Ottocento', 'Novecento.', 'Impresa Porcheddu', 'Esposizione generale italiana', 'Borgo Medievale', 'Vetreria Albano&Macario', 'Terrazza Solferino', 'Mobilificio Torinese F. Cesare Gandolfo', 'Albergo Rocciamelone di', 'Francia', 'Villino Raby', 'Casa Fenoglio-Lafleur', 'Casa Rossi-Galateri', 'Passalacqua', 'Casa Girardi', 'Cibrario 54.', 'Crocetta', 'Casa Maffei']
https://google.it/search?q=Novecento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Italia+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Ottocento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Novecento.+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Impresa+Porcheddu+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Esposizione+generale+italiana+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Borgo+Medievale+Torino&hl=it
Address:  None
Activity:  None
ht

In [5]:
print(spacy.explain("LOC"))
print(spacy.explain("ADP"))

Non-GPE locations, mountain ranges, bodies of water
adposition


In [3]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="PoI_geocoder")

location = geolocator.geocode("Chiesa della Salute", exactly_one = False)

#print(location)
for loc in location: 
    print(loc.address)
    #Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
    print((loc.latitude, loc.longitude))
    print(loc.point)
    #(40.7410861, -73.9896297241625)
    print(loc.raw)
    #{'place_id': '9167009604', 'type': 'attraction', ...}

Chiesa Della Salute, Via Breglio, Borgo Vittoria, Circoscrizione 5, Torino, Piemonte, 10147, Italia
(45.0989972, 7.6796077)
45 5m 56.3899s N, 7 40m 46.5877s E
{'place_id': 12472809, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 1262193690, 'boundingbox': ['45.0989472', '45.0990472', '7.6795577', '7.6796577'], 'lat': '45.0989972', 'lon': '7.6796077', 'display_name': 'Chiesa Della Salute, Via Breglio, Borgo Vittoria, Circoscrizione 5, Torino, Piemonte, 10147, Italia', 'class': 'highway', 'type': 'bus_stop', 'importance': 0.30010000000000003, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/transport_bus_stop2.p.20.png'}


In [4]:
import spacy
from spacy import displacy

nlp = spacy.load('it_core_news_sm')
sentence = "Ai lati della scalinata che conduce all'ingresso della chiesa della Gran Madre di Dio troviamo le due statue raffiguranti la Fede e la Religione, tra le quali si troverebbe il nascondiglio del Graal. La Fontana del Frejus di Piazza Statuto fu ideata dal conte Marcello Panissera per ricordare l'inaugurazione dell'omonimo traforo ed è indicata dagli esoteristi come il \"cuore nero\" della città, per due motivi: perché si trova ad occidente, e quindi in posizione infausta a causa del tramonto del sole, e perché qui vi era la vallis occisorum , luogo di uccisione e di sepoltura."
                                  
doc = nlp(sentence)

displacy.render(doc, style="dep", jupyter=True)

displacy.render(doc, style="ent", jupyter=True)
                